In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math

In [ ]:
Keywords = [
    'teamId',
    'summonerName',
    'individualPosition',
    'lane',
    'championId',
    'championName',
    'champLevel',
    
    ###### KDA
    'kills',
    'deaths',
    'assists',

    ###### Pings
    'commandPings',
    'assistMePings',
    'enemyMissingPings',
    'getBackPings',
    'dangerPings',
    'onMyWayPings',

    ####### vision
    'visionScore',

    #gamedata
    'win'
]

challenge_Keywords = [
    'stealthWardsPlaced',
    'controlWardsPlaced',
]

team_keywords = [
    'baron',
    'dragon',
    'horde',
    'riftHerald',
    'tower'
]

In [ ]:
import mysql.connector
import pandas as pd
eda = mysql.connector.Connect(
    host = "database-1.c96mmei8egml.ap-northeast-2.rds.amazonaws.com",
    port = "3306",
    user = "eda_yc",
    password = "yc",
    database = "eda_project"
)
cursor = eda.cursor()

In [ ]:
cursor.execute("SELECT matchId, teamId, SUM(commandPings), SUM(assistMePings), SUM(enemyMissingPings), SUM(getBackPings), SUM(onMyWayPings), SUM(commandPings)+ SUM(assistMePings)+ SUM(enemyMissingPings)+ SUM(getBackPings)+ SUM(onMyWayPings),win FROM match_data Group By matchId, teamId" )
result = cursor.fetchall()
for result_iterator in result:
    print(result_iterator)

In [ ]:
df = pd.DataFrame(result)
df.columns = ['matchId', 'teamId', 'Ping', 'help', 'mia', 'back', 'going', 'sum', 'win']
df.dropna(axis=0)
df.sort_values('sum', ascending=False)

In [ ]:
df['root'] = df['sum'].astype(float) ** 0.5
df['ping_range'] = df['root'] // 2 * 2
df['ping_range'] = df['ping_range']
graph = df.groupby(['ping_range']).apply(lambda x: x['win'].mean()).reset_index(name='win_rate')

In [ ]:
plt.plot(graph['ping_range'], graph['win_rate'])
plt.title("win rate by Team Pinging")
plt.xlabel("Ping")
plt.ylim(0, 1)
plt.ylabel("win rate")
plt.axhline(y=0.5, color='r', linestyle='--')
plt.axvline(x=10, color='r', linestyle='--')
plt.show()